# Portfolio 5
## IFN646 semester 2 2023
### Arthur Guillaume N11371200


In [259]:
from biom import load_table
import pandas as pd
import numpy as np
import calour as ca
from biom import load_table

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [260]:
meta_file = "metadata.txt"
biom_file = "otu_table.biom"
phylo_file = "97_otu_taxonomy.txt"

We then import the data into calour

In [261]:
%matplotlib notebook

# Task 1

## A )
From Portfolio 4, we were able to detect genes that are differentially expressed (DE) in smokers compared to non-smokers, both in the throat and in the nose. My approach will begin by identifying the bacteria that are differentially expressed exclusively in the throat and in the nose. Once I have the IDs of these bacteria, I will specifically select those features from the metadata table. I will pivot the table to have the sample ID as a column, and each feature will become its own column. For task 4, I will combine the OTU from the nose and from the throat.

The next step will be to incorporate airway and smoker features to filter the data and train a model. Before training the model, I will use lasso regression to reduce the number of features under consideration. As illustrated on page 23 of the Week 8 lecture, I will implement the Partial Least Squares Discriminant Analysis (PLS-DA) algorithm for binary classification (supervised machine learning). We will assess the model on a per-sample basis, and then on a per-patient basis. Since patients have multiple samples, I will group the predictions for a subject and declare a person a smoker if 75% or more of the samples have been predicted as positive. This percentage seems to provide more accurate results compared to using 50% or 100%.

## B) 
When splitting the data into training and testing sets, we need to ensure that the proportion of smokers is the same in both sets by using the stratify feature. As for sample selection, we will only choose samples from the nose or the throat, depending on the task. To make predictions on a subject basis, we will then test the model on the whole dataset, on declare a subject smoker if more than 75% of samples are predicted smoker==yes. For task 3, we will select all the samples for the training. We will use 30% of the dataset as the testing set. After conducting multiple tests, we found that this value provided the best accuracy. Given the relatively small number of samples, we believe it is preferable to have a larger testing set to obtain the most accurate results.

# Task 2

In [262]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from pandas.api.types import CategoricalDtype
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.cross_decomposition import PLSRegression

In [263]:
#build the df
meta_df = pd.read_csv(meta_file,index_col=0, sep="\t")
table = load_table(biom_file)

In [264]:
meta_df.head(4)

,age,airwaysite,antibioticusepast3months,chronic_condition,host_subject_id,respiratorydiseasestatus,sex,sideofbody,smoker,Description
#SampleID,,,,,,,,,,
524.ESC.1.22.NPR,24,Nose,AntibioticUsed.1month,Yes,ESC.1.22,NasalCongestion.mild.current,male,Right,Yes,Nose
524.ESC.1.22.OPL,24,Throat,AntibioticUsed.1month,Yes,ESC.1.22,NasalCongestion.mild.current,male,Left,Yes,Throat
524.ESC.1.22.OPR,24,Throat,AntibioticUsed.1month,Yes,ESC.1.22,NasalCongestion.mild.current,male,Right,Yes,Throat
524.ESC.1.23.NPL,24,Nose,None,Yes,ESC.1.23,NasalCongestion.moderate.current,male,Left,Yes,Nose


In [265]:
#first, check that every sample from the same subject have the same smoker status
nose_df = meta_df[meta_df.airwaysite=='Nose'][['host_subject_id','smoker']]
throat_df = meta_df[meta_df.airwaysite=='Throat'][['host_subject_id','smoker']]
print(f"There are {len(nose_df)} from the nose and {len(throat_df)} from the throat")
#find the patient with only one sample and remove
unique_id_throat = list(set(throat_df.host_subject_id.values))
unique_id_nose = list(set(nose_df.host_subject_id.values))
unique_id_nonsmoker=list(set(meta_df[meta_df.smoker=='No']['host_subject_id'].values))
unique_id_smoker=list(set(meta_df[meta_df.smoker=='Yes']['host_subject_id'].values))
#compare the id of smokers and non smoker
for nonsmoker in unique_id_nonsmoker:
    if nonsmoker in unique_id_smoker:
        print(f'{nonsmoker} is both smoker and non smoker')
for smoker in unique_id_smoker:
    if smoker in unique_id_nonsmoker:
        print(f'{smoker} is both smoker and non smoker')
#now find if everyone has a sample in each airwaysite
for sample in unique_id_throat:
    if sample not in unique_id_nose:
        print(f'{sample} has only a throat sample')
for sample in unique_id_nose:
    if sample not in unique_id_throat:
        print(f'{sample} has only a nose sample')
#everything looks good, we can start the analysis
#map yes to 1 and no to 0
meta_df['smoker'] = meta_df['smoker'].map({'No': 0, 'Yes': 1})

There are 145 from the nose and 146 from the throat


For task 2, using only chronic condition and sex was sufficient and had an 100% accuracy

In [266]:
# Task 3
meta_df.airwaysite.value_counts()

Throat    146
Nose      145
Name: airwaysite, dtype: int64

In [267]:
class MicrobiomeAnalysis:
    
  
    def __init__(self, biom_file, meta_file):
        self.data = ca.read_amplicon(biom_file, meta_file, normalize=10000, min_reads=1000)
        self.biom_table = load_table(biom_file)
        self.meta_table = pd.read_csv(meta_file,index_col=0, sep="\t")
        self.headers = ['ID','Kingdom','Phylum','Class','Order','Family','Genus','Specis']
    def cluster_features(self):
        self.data= self.data.cluster_features()
        
    def create_phylo_table(self, phylo_file):
        self.phylo_table = pd.read_csv(phylo_file,index_col=0,header=None,names=self.headers, sep=";|\t")
        for column in self.headers[1:]:
             self.phylo_table[column] = self.phylo_table[column].str.split('__').str[1]
        ##order by ID for faster search
        self.phylo_table=self.phylo_table.reset_index().sort_values(by='ID').reset_index(drop=True)
        self.phylo_table['Phylum'] = self.phylo_table['Phylum'].apply(lambda x : x[1:-1])
        self.phylo_table['Class'] = self.phylo_table['Class'].apply(lambda x : x[1:-1])
        
    def filter_samples(self, field, values):
        self.data = self.data.filter_samples(field, values)
        
    def filter_abundance(self, treshold):
        self.data = self.data.filter_sum_abundance(treshold)
        
    def perform_differential_abundance(self, feature,group1, group2):
        self.data = self.data.sort_samples(feature)
        self.dd = self.data.diff_abundance(feature, group1, group2,  random_seed=2018)
        self.differentially_expressed_features = self.dd.feature_metadata
        
    
    def get_feature_ids(self):
        # Access the IDs of differentially expressed features
        feature_ids = self.differentially_expressed_features.index
        return [int(feature_id) for feature_id in feature_ids]
    
    def get_taxonomy_answers(self):
        answers = []
       
        
        int_feature_ids = self.get_feature_ids()
        
        for ids in int_feature_ids:
            matching_rows = self.phylo_table[self.phylo_table["ID"] == ids]
            answer = []
            
            if not matching_rows.empty:
                for column in reversed(matching_rows.columns):
                    if(column in ["Specis","Genus" ] or len(answer)==0):
                        if matching_rows.at[matching_rows.index[0], column]:
                            answer.insert(0, matching_rows.at[matching_rows.index[0], column])
                    else:
                        break
            answers.append([ids, " ".join(answer)])
        
        return answers

    def plot_differential_abundance(self, field):
        self.dd.plot(sample_field=field, gui='jupyter')  

In [268]:
##obtain DA fomr samples from the nose
task3_nose = MicrobiomeAnalysis(biom_file, meta_file)
task3_nose.filter_abundance(10)
task3_nose.cluster_features()
task3_nose.filter_samples('Description','Nose')
task3_nose.perform_differential_abundance('smoker', 'Yes', 'No')

task3_nose.create_phylo_table(phylo_file)
task3_answer_nose = task3_nose.get_taxonomy_answers()

2023-10-28 00:11:40 WARNING Found 1 samples that have metadata but do not have data. These samples have been dropped.


C:\Users\lorie\AppData\Local\Temp\ipykernel_4908\2639038977.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.phylo_table = pd.read_csv(phylo_file,index_col=0,header=None,names=self.headers, sep=";|\t")


In [269]:
##obtain DA fomr samples from the throat
task3_throat = MicrobiomeAnalysis(biom_file, meta_file)
task3_throat.filter_abundance(10)
task3_throat.cluster_features()
task3_throat.filter_samples('Description','Throat')
task3_throat.perform_differential_abundance('smoker', 'Yes', 'No')

task3_throat.create_phylo_table(phylo_file)
task3_answer_throat = task3_throat.get_taxonomy_answers()

2023-10-28 00:11:48 WARNING Found 1 samples that have metadata but do not have data. These samples have been dropped.


C:\Users\lorie\AppData\Local\Temp\ipykernel_4908\2639038977.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.phylo_table = pd.read_csv(phylo_file,index_col=0,header=None,names=self.headers, sep=";|\t")


In [270]:
##obtain DA fomr samples from tall samples
task3_smoker = MicrobiomeAnalysis(biom_file, meta_file)
task3_smoker.filter_abundance(10)
task3_smoker.cluster_features()
task3_smoker.perform_differential_abundance('smoker', 'Yes', 'No')

task3_smoker.create_phylo_table(phylo_file)
task3_smoker = task3_smoker.get_taxonomy_answers()

2023-10-28 00:11:56 WARNING Found 1 samples that have metadata but do not have data. These samples have been dropped.


C:\Users\lorie\AppData\Local\Temp\ipykernel_4908\2639038977.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  self.phylo_table = pd.read_csv(phylo_file,index_col=0,header=None,names=self.headers, sep=";|\t")


In [271]:
#find differences between nose and throat
nose_bact_diff = []
throat_bact_diff =[]


for bact in task3_answer_nose:
    if bact not in task3_answer_throat:
        nose_bact_diff.append(bact[0]) 
for bact in task3_answer_throat:
    if bact not in task3_answer_nose:
        throat_bact_diff.append(bact[0])
print(f"Throat: {len(throat_bact_diff)} DA")
print(f"Nose: {len(nose_bact_diff)} DA")


Throat: 54 DA
Nose: 126 DA


## Task 2

In [272]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

def doPCASVMClassification(meta_table, bact_diff, number_components, filter=False, localisation='default'):
    number_components = number_components
#now create a table with the samples as index, the identified genes as column, and add the smoker target
    # Load the Biom table
    biom_table = load_table(biom_file)


    # Convert Biom table to a DataFrame
    data = biom_table.to_dataframe()

    # Ensure the index is in integer format and sorted
    data.index = data.index.astype(int)
    data.sort_index(ascending=True, inplace=True)
 
    # Filter data based on 'nose_bact_diff'
    data= data[data.index.isin(bact_diff)]
    # Transpose and reset the index to have samples as rows
    pivot = data.transpose().reset_index()
    pivot.rename(columns={'index': 'samples'}, inplace=True)
    # Add 'smoker',and airwaysite information from the metadata table
    pivot['smoker'] = pivot['samples'].map(meta_table['smoker'])
    pivot['airwaysite'] = pivot['samples'].map(meta_table['airwaysite'])
    pivot['host_subject_id'] = pivot['samples'].map(meta_table['host_subject_id'])
   


    
    pivot_filtered = pivot.copy()
    #filter for nose only
    if(filter and localisation!='default'):
        pivot_filtered = pivot_filtered[pivot_filtered['airwaysite']==localisation]
    X= pivot_filtered.drop(['smoker', 'samples', 'airwaysite','host_subject_id'], axis=1)
    X.columns = X.columns.astype(str)
    features_cols = list(X.columns)
    X= np.array(X)
    #scale the data
    scaler = preprocessing.StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    y= pivot_filtered.smoker.values
    #feature resuction with lasso, first step is to find the best value for alpha; we are not interested in the absolute best result, so we won't look for convergence
    lasso_cv = LassoCV(alphas=np.logspace(-6, 6, 100), cv=5)  
    lasso_cv.fit(X_scaled, y)
    #get the best alha from cross validation
    best_alpha = lasso_cv.alpha_
    #create the model
    lasso_best = Lasso(alpha=best_alpha)
    lasso_best.fit(X_scaled, y)
    #selected the features with a non 0 lasso coef from X
    X_selected = X_scaled[:, np.abs(lasso_best.coef_) > 0]
    print(f"number of feature used:{len(X_selected[0])}")
    
    #split the data
    X_train, X_test, y_train, y_test = train_test_split(np.array(X_selected), y, test_size=0.3, random_state=42, stratify=y)
    #Now build a model
    pls_binary = PLSRegression(n_components=number_components)
    # Fit the training set
    pls_binary.fit(X_train, y_train)
    # "Force" binary prediction by thresholding
    binary_prediction = (pls_binary.predict(X_test)[:,0] > 0.5).astype('uint8')
    binary_prediction_train = (pls_binary.predict(X_train)[:,0] > 0.5).astype('uint8')
    #test the model on the unfilted dataset
    subject_id = pivot[['smoker', 'host_subject_id']]
    X_all= pivot.drop(['smoker', 'samples', 'airwaysite','host_subject_id'], axis=1)
    X_all.columns = X_all.columns.astype(str)
    X_all= np.array(X_all)
    #scale the data
    X_all = scaler.transform(X_all)
    
    X_all_selected = X_all[:, np.abs(lasso_best.coef_) > 0]


    
    y_all= pivot.smoker.values
    binary_prediction_all = (pls_binary.predict(X_all_selected)[:,0] > 0.5).astype('uint8')
   
    #now for each subject ID, check if at least 1 is 0. To do that, groupby id, compute mean set to 1 of >0.75
    subject_id=subject_id.assign(predicted=binary_prediction_all)
    subject_id = subject_id.groupby('host_subject_id').mean()
    subject_id.predicted = subject_id.predicted.apply(lambda x : 0 if x < 0.75 else 1)
    y_pred_agg=np.array(subject_id.predicted.values)
    y_real =np.array(subject_id.smoker.values)
  
    
    #evaluation
    accuracy_all = accuracy_score(y_real,y_pred_agg)
    accuracy = accuracy_score(y_test, binary_prediction)
    accuracy_train = accuracy_score(y_train, binary_prediction_train)
    report = classification_report(y_test, binary_prediction)
    report_all = classification_report(y_real,y_pred_agg)
    print(f'Accuracy  test {localisation}: {accuracy}')
    print(f'Accuracy  train {localisation}: {accuracy_train}')
    print(f'Confusion table for test on sample from {localisation}')
    print(report)
   
    print(f'Accuracy all dataset: {accuracy_all}')
    print(f'Confusion table for test on all samples, subject classification')
    print(report_all)


## Task 2

In [273]:
doPCASVMClassification(meta_df,nose_bact_diff, 1, filter=True, localisation='Nose')

number of feature used:32
Accuracy  test Nose: 0.8863636363636364
Accuracy  train Nose: 0.8415841584158416
Confusion table for test on sample from Nose
              precision    recall  f1-score   support

           0       0.95      0.82      0.88        22
           1       0.84      0.95      0.89        22

    accuracy                           0.89        44
   macro avg       0.89      0.89      0.89        44
weighted avg       0.89      0.89      0.89        44

Accuracy all dataset: 0.821917808219178
Confusion table for test on all samples, subject classification
              precision    recall  f1-score   support

         0.0       0.90      0.72      0.80        36
         1.0       0.77      0.92      0.84        37

    accuracy                           0.82        73
   macro avg       0.83      0.82      0.82        73
weighted avg       0.83      0.82      0.82        73



Using only samples from the nose for the Discriminant Analysis (DA) and subsequently for training and testing the model, we achieved an accuracy of 0.89. We utilized a single component, and out of 126 features, 32 were selected for sparse PLS. Among all the samples from the nose, we correctly identified 95% of the smokers and 83 of the non-smokers, with our predictions for smokers being accurate 84% of the time. The fact that the testing accuracy is higher than the training accuracy suggests there is no overfitting.

We then tested our model on the entire dataset, grouping results when multiple samples were available. The accuracy reduced to 82%, and the precision for smokers dropped significantly to 77%. Overall, the model had an average precision of 83% and recall of 82%. 

## Task 3

In [274]:
doPCASVMClassification(meta_df,throat_bact_diff,1, filter=True, localisation='Throat')

number of feature used:29
Accuracy  test Throat: 0.75
Accuracy  train Throat: 0.8811881188118812
Confusion table for test on sample from Throat
              precision    recall  f1-score   support

           0       0.76      0.73      0.74        22
           1       0.74      0.77      0.76        22

    accuracy                           0.75        44
   macro avg       0.75      0.75      0.75        44
weighted avg       0.75      0.75      0.75        44

Accuracy all dataset: 0.821917808219178
Confusion table for test on all samples, subject classification
              precision    recall  f1-score   support

         0.0       0.81      0.83      0.82        36
         1.0       0.83      0.81      0.82        37

    accuracy                           0.82        73
   macro avg       0.82      0.82      0.82        73
weighted avg       0.82      0.82      0.82        73



Using only samples from the throat for the Discriminant Analysis (DA) and then for training, we achieved an accuracy of 0.75. We utilized one component, and out of 54 features, 29 were selected. Among all the samples from the throat, we correctly identified 77% of the smokers and 73 of the non-smokers, with our predictions for smokers being correct 74% of the time. The fact that the training accuracy is higher than the testing accuracy indicates some overfitting.

When deploying our model on a per-patient basis, the accuracy increased to 82%. Results were comparable to the model trained on samples from the nose but with lower recall for positive results and a higher recall for negative results. Overall precision and recall were similar to the results obtained with the model trained using samples from the nose.

## Task 4

In [275]:
#combine all OTUs DA
all_out = throat_bact_diff+nose_bact_diff
#here the 'NOse' parameter is a something I need to clean in my code, it does nothing here , we take all the samples
doPCASVMClassification(meta_df, all_out, 1, filter=False)

number of feature used:63
Accuracy  test default: 0.7471264367816092
Accuracy  train default: 0.8226600985221675
Confusion table for test on sample from default
              precision    recall  f1-score   support

           0       0.70      0.86      0.77        43
           1       0.82      0.64      0.72        44

    accuracy                           0.75        87
   macro avg       0.76      0.75      0.74        87
weighted avg       0.76      0.75      0.74        87

Accuracy all dataset: 0.8082191780821918
Confusion table for test on all samples, subject classification
              precision    recall  f1-score   support

         0.0       0.72      1.00      0.84        36
         1.0       1.00      0.62      0.77        37

    accuracy                           0.81        73
   macro avg       0.86      0.81      0.80        73
weighted avg       0.86      0.81      0.80        73



Using all the samples and combining the OTUs from both the nose and the throat, we achieved an accuracy of 75%. However, we observed some overfitting, as indicated by a training accuracy of 82%. The model was conservative, detecting only 64% of the smokers, although with a precision of 82%. These results were worse than the test results from the model trained on either the nose or the throat alone on the entire dataset, but the precision for positive results was similar (77% for the nose model and 83% for the throat). The recall for positive results was also lower for both models, at 64% compared to 92% and 81% for the nose and throat, respectively. We still observed a fair amount of overfitting.

However, when we moved from the sample level to the subject level, the results were decent, with an accuracy of 81%. The model made no errors in positive classification, but 38% of the positives were misclassified. The overall recall of 81% and precision were similar to the model trained on the throat or nose, but the precision was better, at 86% compared to 83% or 82% for the nose or the throat, respectively